In [380]:
txt = "Hello SDw:sd, and the in on not asda d. adf! 989.09 6:09 adadsa-adas."
txt2 = "Hello SDw:sd, asda d. adf! 989.09 6:09 adadsa-adas.Hello SDw:sd, asda d. adf! 989.09 6:09 adadsa-adas."
txt3 = "Hello SDw:sd, asda d. adf! 989.adas.Hello SDw:sd, asda d. adf! 989.09 6:09 adadsa-adas."
txt4 = "Hello SDw:sd, asda d. a"
txt5 = "Hello SDw:sd, asda d. adf! 989.09 6:09 adadsa-adas.Hello SDw:sd, asda d. adf! 989.09 6:09 adadsa-adas.Hello SDw:sd, asda d. abs iojknas sdfdsd"
txt10 = "Hello SDw:sd, asda d. adf! 9df frfw3 as 1 -adas.Hello SDw:sd, asda d. adf! 989.09 6:09 adadsa-adas.Hello SDw:sd, asda d. abs"

docs = {"d1": txt, "d2": txt2, "d3": txt3, "d4": txt4, "d5": txt5}
docs2 = {"d6": txt, "d7": txt2, "d8": txt3, "d9": txt4, "d10": txt10}

# TODO: loadTermMap() and docMap
termMap = []
docMap = []
catalog = defaultdict(int)

merge_count = 0
doc_count = 0

In [381]:
import re
import nltk
from nltk.stem.lancaster import LancasterStemmer
from nltk.corpus import stopwords
from collections import defaultdict, OrderedDict


stopwords = set(stopwords.words('english'))

In [382]:
def tokenizer(text):
    return list(filter(lambda x: x != "", re.compile(r'[\s,!-\.]').split(text.lower())))

def stemmer(term):
    stemmer = LancasterStemmer()
    return stemmer.stem(term)

def stopWordsRemoval(tokens):
    new_tokens = []
    for token in tokens:
        if token not in stopwords:
            new_tokens.append(token)
    return new_tokens

def preprocess(text):
    tokens = tokenizer(text)
    stokens = map(stemmer, tokens)
    return stopWordsRemoval(stokens)

In [383]:
def getSetItemId(term, someMap):
    if term not in someMap:
        i = len(someMap)
        someMap.append(term)
    else:
        i = someMap.index(term)
    return i

def getItemName(id, someMap):
    return someMap[id]

assert(getSetItemId("hello", [])) == 0
assert(getSetItemId("d1", [])) == 0

assert(getItemName(0, ["hello"])) == "hello" 

In [384]:
def createTermList(docs):
    itermCount = defaultdict(dict)
    df = defaultdict(int)
    ttf = defaultdict(int)

    for doc in docs:
        doc_tokens = preprocess(docs[doc])
        for token in enumerate(doc_tokens):
            pos = token[0]
            tokenId = getItemId(token[1], termMap)
            docId = getItemId(doc, docMap)

            if tokenId not in itermCount.keys():
                itermCount[tokenId] = {}
            if docId not in itermCount[tokenId].keys():
                itermCount[tokenId][docId] = {}
                itermCount[tokenId][docId]['tf'] = 0
                itermCount[tokenId][docId]['pos'] = []
            itermCount[tokenId][docId]['tf'] += 1
            itermCount[tokenId][docId]['pos'].append(pos)
    for tokenId, docs in itermCount.items():
        df[tokenId] = len(docs.keys())
    for tokenId, docs in itermCount.items():
        ttf[tokenId] = sum(map(lambda d: d['tf'], docs.values()))
    return (itermCount, df, ttf)

In [385]:
def writeTermFile(filename, itermCount, df, ttf):
    catalog = defaultdict(int)
    with open(filename, "w+") as out:
        for termId, docs in itermCount.items():
            catalog[termId] = out.tell()
            outStr = str(termId) + ":" + str(df[termId]) + ":" + str(ttf[termId]) + ":" 
            tempStr = []
            for docId, values in docs.items():
                tempStr.append(str(docId) + "," + str(values['tf']) + "," + "{}".format(".".join(map(lambda x: str(x), values['pos']))))
            outStr += "|".join(tempStr)
            print(outStr)
            out.write(outStr+"\n")
    return catalog

def writeCatalog(filename, catalog):
    with open(filename, "w+") as out:
        for termId, offset in catalog.items():
            out.write(termId + "," + offset + "\n")

In [386]:
def readTermFromFile(filename, termIdRef, catalog):
    ntc = {}
    f = open(filename)
    f.seek(catalog[termIdRef])
    line = f.readline().strip()
    (termId, df, ttf, docs) = line.strip().split(":")
    mdocs = docs.split("|")
    ntc= {}
    for p in mdocs:
        docId, tf, els = p.split(",")
        ntc[docId] = {}
        ntc[docId]['tf'] = int(tf) 
        ntc[docId]['pos'] = list(map(lambda x: int(x), els.split("."))) 
    return (ntc, int(df), int(ttf))

def readCatalog(catalogFile):
    catalog = {}
    # Read from file
    with open(catalogFile, "r") as infile:
        for line in infile:
            termId, offset = line.split(",")
            catalog[termId] = offset
    return catalog

In [387]:
(itermCount1, df1, ttf1) = createTermList(docs)
(itermCount2, df2, ttf2) = createTermList(docs2)

In [388]:
doc_count += 1
file_name1 = "test{}.index".format(doc_count)
doc_count += 1
file_name2 = "test{}.index".format(doc_count)
catalog1 = writeTermFile(file_name1, itermCount1, df1, ttf1)
catalog2 = writeTermFile(file_name2, itermCount2, df2, ttf2)
print(file_name1, file_name2)

0:5:9:0,3,0.9.18|1,1,0|2,2,0.6|3,1,0|4,2,0.9
1:5:9:0,3,1.10.19|1,1,1|2,2,1.7|3,1,1|4,2,1.10
2:5:9:0,3,2.11.20|1,1,2|2,2,2.8|3,1,2|4,2,2.11
3:4:7:0,2,3.12|1,1,3|2,2,3.9|4,2,3.12
4:4:7:0,2,4.13|1,1,4|2,2,4.10|4,2,4.13
5:4:6:0,2,5.14|1,1,5|2,1,11|4,2,5.14
6:4:6:0,2,6.15|1,1,6|2,1,12|4,2,6.15
7:4:6:0,2,7.16|1,1,7|2,1,13|4,2,7.16
8:4:7:0,2,8.17|1,1,8|2,2,5.14|4,2,8.17
9:1:1:0,1,21
10:1:1:0,1,22
11:1:1:0,1,23
0:5:9:8,3,0.8.17|9,2,0.6|5,2,0.9|6,1,0|7,1,0
1:5:9:8,3,1.9.18|9,2,1.7|5,2,1.10|6,1,1|7,1,1
2:5:9:8,3,2.10.19|9,2,2.8|5,2,2.11|6,1,2|7,1,2
3:4:7:8,2,3.11|9,2,3.9|5,2,3.12|7,1,3
4:4:6:8,1,12|9,2,4.10|5,2,4.13|7,1,4
5:4:5:8,1,13|9,1,11|5,2,5.14|7,1,5
6:4:5:8,1,14|9,1,12|5,2,6.15|7,1,6
7:4:5:8,1,15|9,1,13|5,2,7.16|7,1,7
8:4:7:8,2,7.16|9,2,5.14|5,2,8.17|7,1,8
9:1:1:8,1,20
12:1:1:8,1,4
13:1:1:8,1,5
14:1:1:8,1,6
test1.index test2.index


In [389]:
merge(catalog1, catalog2, file_name1, file_name2, merge_count)

0:10:18:0,3,0.9.18|8,3,0.8.17|9,2,0.6|2,2,0.6|5,2,0.9|4,2,0.9|3,1,0|7,1,0|6,1,0|1,1,0
1:10:18:0,3,1.10.19|8,3,1.9.18|9,2,1.7|2,2,1.7|5,2,1.10|4,2,1.10|3,1,1|7,1,1|6,1,1|1,1,1
2:10:18:0,3,2.11.20|8,3,2.10.19|9,2,2.8|2,2,2.8|5,2,2.11|4,2,2.11|3,1,2|7,1,2|6,1,2|1,1,2
3:8:14:9,2,3.9|2,2,3.9|0,2,3.12|8,2,3.11|5,2,3.12|4,2,3.12|7,1,3|1,1,3
4:8:13:9,2,4.10|2,2,4.10|0,2,4.13|5,2,4.13|4,2,4.13|8,1,12|7,1,4|1,1,4
5:8:11:0,2,5.14|5,2,5.14|4,2,5.14|9,1,11|2,1,11|8,1,13|7,1,5|1,1,5
6:8:11:0,2,6.15|5,2,6.15|4,2,6.15|9,1,12|2,1,12|8,1,14|7,1,6|1,1,6
7:8:11:0,2,7.16|5,2,7.16|4,2,7.16|9,1,13|2,1,13|8,1,15|7,1,7|1,1,7
8:8:14:9,2,5.14|2,2,5.14|0,2,8.17|8,2,7.16|5,2,8.17|4,2,8.17|7,1,8|1,1,8
9:2:2:8,1,20|0,1,21
10:1:1:0,1,22
11:1:1:0,1,23
12:1:1:8,1,4
13:1:1:8,1,5
14:1:1:8,1,6


('merge1.index',
 defaultdict(int,
             {0: 0,
              1: 86,
              2: 175,
              3: 265,
              4: 336,
              5: 407,
              6: 474,
              7: 541,
              8: 608,
              9: 681,
              10: 701,
              11: 715,
              12: 729,
              13: 742,
              14: 755}))

In [390]:
def mergeDocs(termId, filename, catalog, n_docs, df1, ttf1):
    merged_termCount = {}
    df_merge = df1
    ttf_merge = ttf1
#     print(n_docs)
    old_docs, df2, ttf2 = readTermFromFile(filename, termId, catalog)
#     print(old_docs)
    join_docs = dict()
    join_docs.update(n_docs)
    join_docs.update(old_docs)
    newdocs = OrderedDict(sorted(join_docs.values(), key=lambda x: x["tf"], reverse=True))

    merged_termCount[termId] = newdocs
    df_merge += int(df2)
    ttf_merge += int(ttf2)
        
    return (newdocs, df_merge, ttf_merge)

In [391]:
def writeTempData(itermCount, df, ttf):
    filename = "temp.index"
    with open(filename, "a") as out:
        for termId, docs in itermCount.items():
            if termId in catalog.keys():
                res = mergeDocs(termId, filename, catalog, docs, df[termId], ttf[termId])
                docs = res[0]
                df[termId] = res[1]
                ttf[termId] = res[2]
            catalog[termId] = out.tell()
            outStr = str(termId) + ":" + str(df[termId]) + ":" + str(ttf[termId]) + ":" 
            tempStr = []
            for docId, values in docs.items():
                tempStr.append(str(docId) + "," + str(values['tf']) + "," + "{}".format(".".join(map(lambda x: str(x), values['pos']))))
            outStr += "|".join(tempStr)
            print(outStr)
            out.write(outStr+"\n")
    return catalog

def writeCatalog(filename, catalog):
    with open(filename, "w+") as out:
        for termId, offset in catalog.items():
            out.write(termId + "," + offset + "\n")

In [392]:
writeTempData(itermCount1, df1, ttf1)

0:5:9:0,3,0.9.18|1,1,0|2,2,0.6|3,1,0|4,2,0.9
1:5:9:0,3,1.10.19|1,1,1|2,2,1.7|3,1,1|4,2,1.10
2:5:9:0,3,2.11.20|1,1,2|2,2,2.8|3,1,2|4,2,2.11
3:4:7:0,2,3.12|1,1,3|2,2,3.9|4,2,3.12
4:4:7:0,2,4.13|1,1,4|2,2,4.10|4,2,4.13
5:4:6:0,2,5.14|1,1,5|2,1,11|4,2,5.14
6:4:6:0,2,6.15|1,1,6|2,1,12|4,2,6.15
7:4:6:0,2,7.16|1,1,7|2,1,13|4,2,7.16
8:4:7:0,2,8.17|1,1,8|2,2,5.14|4,2,8.17
9:1:1:0,1,21
10:1:1:0,1,22
11:1:1:0,1,23


defaultdict(int,
            {0: 407,
             1: 452,
             2: 499,
             3: 546,
             4: 584,
             5: 623,
             6: 660,
             7: 697,
             8: 734,
             9: 773,
             10: 786,
             11: 800})

In [393]:
writeTempData(itermCount2, df2, ttf2)

TypeError: string indices must be integers

In [ ]:
# class DocTerm:
#     def __init__(self, tf, pos):
#         self.tf = tf
#         self.pos = pos

In [ ]:
# termCount = {}
# for doc in docs:
#     termCount[doc] = defaultdict(int)
# termCount

# docs = docs2

In [ ]:
# BasicLoadInvList(term):
# 	off, length from catalog
# 	move invfilecounter to offset
# 	read length of the term
# 	parse the invertlst into a local DS

In [ ]:
# Load both catalogs in the memory
# Pick first term in invfile 1
# 	invList_term1 = load invfile for offset, length of term 1
# 	scan catalog2 for term2
# 	if yes:
# 		get invList_term2 = load from file2
# 		join the two lists
# 	write (inv_combined)
# 	write (catalog_combined)

# Sort the docs in decreasing order of tf count

In [342]:
def merge(catalog1, catalog2, file1, file2, merge_count):
    merged_termCount = {}
    df_merge = {}
    ttf_merge = {}

    common_terms = set(catalog1.keys()).intersection(set(catalog2.keys()))
    # print(list(common_terms))
    unique_terms = set(catalog1.keys()).union(set(catalog2.keys())) - common_terms
    # print(unique_terms)

    for termId in common_terms:
        temp_docs1, df1, ttf1 = readTermFromFile(file1, termId, catalog1)
        temp_docs2, df2, ttf2 = readTermFromFile(file2, termId, catalog2)
        join_docs = dict()
        join_docs.update(temp_docs1)
        join_docs.update(temp_docs2)
        newdocs = OrderedDict(sorted(join_docs.items(), key=lambda x: x[1]["tf"], reverse=True))

        merged_termCount[termId] = newdocs
        df_merge[termId] = int(df1) + int(df2)
        ttf_merge[termId] = int(ttf1) + int(ttf2)

    for termId in set(catalog1.keys()).difference(set(catalog2.keys())):
        temp_docs, df, ttf = readTermFromFile(file1, termId, catalog1)
        merged_termCount[termId] = temp_docs
        df_merge[termId] = df
        ttf_merge[termId] = ttf

    for termId in set(catalog2.keys()).difference(set(catalog1.keys())):
        temp_docs, df, ttf = readTermFromFile(file2, termId, catalog2)
        merged_termCount[termId] = temp_docs
        df_merge[termId] = df
        ttf_merge[termId] = ttf
        
    merge_count += 1
    merge_index_name = "merge"+str(merge_count)+".index"
    merge_catalog = writeTermFile(merge_index_name, merged_termCount, df_merge, ttf_merge)
    return (merge_index_name, merge_catalog)